# Approximations

This notebook is on approxiamtions of systems.
The balanced truncation algorithm is implemented in the library.

The idea of balanced truncation is the idea that in the balanced represantation, we can map each state to a certain singular value of the Hankel matrix.
Using this we can remove the states that correspond to singular values $\sigma_i < \epsilon$ for some approxiamtion parameter $\epsilon$. 

In the library this is done by converting the system to a ordered representation. This is a represantation, that is balanced and the states are ordered, such that the accoring singular values are in decreasing order.
When this is done, the singular values of the Hankel operators are computed and stored.
Later the states can be removed for different $\epsilon$s.

In [ ]:
import numpy as np
from tvsclib.canonical_form import CanonicalForm
from tvsclib.mixed_system import MixedSystem
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.approximation import Approximation
import matplotlib.pyplot as plt
import tvsclib.utils as utils
import tvsclib.math as math
import tvsclib.identification as ident

## Introductary example
First we consider a simple case with only two stages. In this case we only have one Hankel matrix for the causal and anticausal part each.
Here we design a matrix such that the Hankel matrices have certian $\sigma$s

In [ ]:
#create a special matrix 
n = 10
T = np.random.rand(2*n,2*n)
Uc, s, Vch = np.linalg.svd(T[n:,:n], full_matrices=True)
Ua, s, Vah = np.linalg.svd(T[:n,n:], full_matrices=True)

sc = np.linspace(5,0,n)
sa = np.linspace(5,0,n)

#sc = np.zeros(n)
#sa = np.zeros(n)
#sc[0]=5
#sc[1]=5
#sa[0]=5
#sa[1]=0

T[n:,:n] = Uc*sc@Vch
T[:n,n:] = Ua*sa@Vah

TOs = ToeplitzOperator(T, [n,n], [n,n])
S = SystemIdentificationSVD(TOs,epsilon=1e-10)
system = MixedSystem(S)

utils.show_system(system)
plt.clim(-1.5,1.5)

Now we create an approxiamtion object for a system.

With the approximation objectm we can access the sigmas of the system:

In [ ]:
approx =Approximation(system)
plt.plot(np.arange(len(approx.sigmas_causal[0]))+1,approx.sigmas_causal[0],'x',label="causal")
plt.plot(np.arange(len(approx.sigmas_anticausal[0]))+1,approx.sigmas_anticausal[0],'+',label="anticasual")
plt.axhline(3,label=r"$\epsilon$")
plt.xlabel("i")
plt.ylabel(r"$\sigma_i$")
plt.legend()

Now create a approxiamtion for a certain $\epsilon$:

In [ ]:
eps = 3

approx_system=approx.get_approxiamtion(eps)
utils.show_system(approx_system)
plt.clim(-1.5,1.5)

We can now see that the number of states is reduced. we have as many states as we have $\sigma_i > \epsilon$

In [ ]:
print(approx_system)

In [ ]:
#check the system by creating a reference matrix
mat_ref = T.copy()
nc = np.count_nonzero(sc>eps)
na = np.count_nonzero(sa>eps)

mat_ref[n:,:n] = Uc[:,:nc]*sc[:nc]@Vch[:nc,:]
mat_ref[:n,n:] = Ua[:,:na]*sa[:na]@Vah[:na,:]

dif = mat_ref-approx_system.to_matrix()
print(np.max(abs(dif))<1e-13)

For this simple case we can compute the approxiamtion error exactly for different norms:
Here we consider the Frobenius norm $\|X\|_2$, the Operator norm  (Spectral norm) $\|X\|$ and the Hankel Norm $\|X\|_H$

The Hankel Norm is defined as the supremum over the Operator norm of each individual Hankel matrix. $$\|T\|_H = \sup_{i}\|T_i\|$$

These norms all depend on the singular values.
If we have a system with two stages the singular values of the difference $T-\tilde{T}$ are the singular values that got removed during the approximation.

Analogously, the singular values of the Hankel matricies of $T-\tilde{T}$ are again the removed singular values.

In [ ]:
delta = T-approx_system.to_matrix()

print("Frobenius norm:")
print("Prediction: ",np.sqrt(np.sum(sc[sc<eps]**2)+np.sum(sa[sa<eps]**2)))
print("Actual:     ",np.linalg.norm(delta))

print("Operator norm:")
print("Prediction: ",max(np.max(sc[sc<eps]),np.max(sa[sa<eps])))
print("Actual:     ",np.linalg.norm(delta,ord=2))

print("Hankel norm:")
print("Prediction: ",max(np.max(sc[sc<eps]),np.max(sa[sa<eps])))
print("Actual:     ",math.hankelnorm(delta,2*[10],2*[10]))

## More complex system

Usually the systems have more stages. Here it is no longer possible to compute the approxiamtion error, but it can be bounded for the Frobenius and Operator norm.

For these Norms we can bound the total approxiamtion error to the sum of the approxiamtion errors for the different stages.

For the Operator norm this reduces to the simple bound

$$\|T-\tilde{T}\| \leq (K-1)\epsilon$$

In [ ]:
dims_in =  [10]*8
dims_out = [10]*8
matrix = np.random.rand(sum(dims_out), sum(dims_in))

system = MixedSystem(S)

print(system)

In [ ]:
dims_in =  [10]*8
dims_out = [10]*8
T = np.random.rand(sum(dims_out), sum(dims_in))
system = ident.identify(T, dims_in, dims_out,epsilon=1e-14)

print(system)

In [ ]:
eps = 2
approx =Approximation(system)
system_approx = approx.get_approxiamtion(eps)
print(system_approx)

The cost for computing the system now has decreased

In [ ]:
print("Original:    ",system.cost())
print("Approximated:",system_approx.cost())

But this has resulted in an approxiamtion error

In [ ]:
delta = T-system_approx.to_matrix()

print("Frobenius norm:")
print("Bound:      ",sum([np.sqrt(np.sum(sc[sc<eps]**2)+np.sum(sa[sa<eps]**2)) for (sc,sa) in zip(approx.sigmas_causal,approx.sigmas_anticausal)]))
print("Actual:     ",np.linalg.norm(delta))

print("Operator norm:")
print("Bound:      ",eps*(len(system.causal_system.stages)-1))
print("Actual:     ",np.linalg.norm(delta,ord=2))

Now we Obtain the hankel norm of the difference. We will see that $\|T-\tilde{T}\|_H$ is often slighly bigger than $\epsilon$. 

In [ ]:
hankelnorm(system_approx.to_matrix()-system.to_matrix(),dims_in,dims_out)

In [ ]:
#get system using identifucation
S_appr = SystemIdentificationSVD(T,epsilon=eps,relative=False)
system_approx_ident = MixedSystem(S_appr)
print(system_approx_ident)

In [ ]:
np.max(abs(system_approx_ident.to_matrix()-system_approx.to_matrix()))

In [ ]:
hankelnorm(system_approx_ident.to_matrix()-system.to_matrix(),dims_in,dims_out)

Note here: It is also possible to obtain more genreal Hankel norm approxiamtions. These sattsify the condition:

$$\| \Gamma^{-1}(T-\tilde{T})\|_H \leq 1 $$

Details on these can be found in [[1](#Dewilde_1998)]

[1]:<a id='Dewilde_1998'></a> P. Dewilde and A.-J. van der Veen. Time-Varying Systems and Computations. Jan. 1, 1998. ISBN : 978-0-7923-8189-1. [DOI : 10.1007/978- 1- 4757-
2817-0](https://doi.org/10.1007/978-1-4757-2817-0)